# <font color='056938'> **Librerias** </font>

In [1]:
#instalar lightfm para sistema de recomendación de filtro colaborativo
!pip install lightfm

In [2]:
from google.colab import drive
import sys
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
from ipywidgets import interact, Dropdown, IntSlider
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn import set_config
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

In [3]:
drive.mount('/content/drive')

#Define la parte del directorio que quieres trabajar
path = "/content/drive/MyDrive/Mod2/ANALITICA3"

sys.path.append(path)##para importar archivo de funciones propias a traves de import
os.chdir(path)## para que por defecto suba y descargue archivos partiendo de esa ruta
sys.path.append(f"{path}") #agragarle al path, poder leer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='056938'> **Cargar base de datos** </font>

In [4]:
con=sql.connect("data//db_movies")# conectarse a base de datos existente
cur=con.cursor()#otro tipo de conexión para ejecutar consultas en la base de datos sin traer ni llevar información

In [5]:
cur.execute("""SELECT name FROM sqlite_master WHERE type='table'""")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [6]:
tabla_completa = pd.read_sql("SELECT * FROM full_ratings", con)
tabla_completa.head()

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995


# <font color='056938'> **Sistema de recomendación filtro colaborativo basado en usuario** </font>

Este sistema consiste en comparar el historial de interacciones de calificaciones del usuario objetivo con el de otros usuarios. El objetivo es que una vez identificados los usuarios más parecidos, el sistema recomiende peliculas que esos usuarios han calificado bien y que el usuario objetivo aún no ha visto.

Para esto, se construye un **pipeline** que contiene los siguientes pasos:

**Paso 1.Crear dataset de train y test:** Se crea una función crearDataset que divide los datos originales en conjuntos de entrenamiento y prueba, y prepara los objetos Dataset necesarios para la creación de interacciones.

**Paso 2.Crear matriz usurio movie:**  Se crea una función Matriz_usurio_Movie que transforma los datos en matrices de interacciones y pesos que el modelo puede utilizar.

**Paso 3. Crear modelo de LightFM:** Se crea una función el EntrenarModelo que entrena un modelo LightFM con la función de pérdida especificada durante un número definido de épocas, y calcula el AUC para evaluar su desempeño tanto en entrenamiento como en prueba.

In [7]:
# Paso 1: Función para crear dataset de train y test
class crearDataset(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):

        # Crear dataset_train y dataset_test con los mismos usuarios y elementos
        self.dataset_train = Dataset()
        self.dataset_test = Dataset()

        all_unique_users = X['user_id'].unique()
        all_unique_items = X['movie_id'].unique()

        # Ajustar los datasets para entrenamiento y prueba
        self.dataset_train.fit(users=all_unique_users, items=all_unique_items)
        self.dataset_test.fit(users=all_unique_users, items=all_unique_items)

        # se divide el DataFrame original en train_df y test_df usando train_test_split
        self.train_df, self.test_df = train_test_split(X, test_size=0.2, random_state=42)

        return self

    def transform(self, X):
        # Retornar los datos procesados: train_df, test_df y los datasets
        return {
            'dataset_train': self.dataset_train,
            'dataset_test': self.dataset_test,
            'train_df': self.train_df,
            'test_df': self.test_df,
        }


In [8]:
#Paso 2. Función para crear matriz usurio movie
class Matriz_usurio_Movie(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        train_df = X['train_df']
        test_df = X['test_df']
        dataset_train = X['dataset_train']
        dataset_test = X['dataset_test']

        train_interactions_list = [
            (row['user_id'], row['movie_id'], row['movie_rating']) for _, row in train_df.iterrows()
        ]
        test_interactions_list = [
            (row['user_id'], row['movie_id'], row['movie_rating']) for _, row in test_df.iterrows()
        ]

        train_interactions, train_weights = dataset_train.build_interactions(train_interactions_list)
        test_interactions, test_weights = dataset_test.build_interactions(test_interactions_list)

        return {
            'train_interactions': train_interactions,
            'test_interactions': test_interactions,
            'train_weights': train_weights,
            'test_weights': test_weights,
            'dataset_train': dataset_train,
            'dataset_test': dataset_test
        }


In [9]:
#Paso 3. Función para crear modelo de LightFM:
class EntrenarModelo(BaseEstimator, TransformerMixin):
    def __init__(self, loss, epochs, random_state=42):
        self.loss = loss
        self.epochs = epochs
        self.random_state = random_state
        self.train_auc = None
        self.test_auc = None

    def fit(self, data, y=None):
        self.train_interactions = data['train_interactions']
        self.test_interactions = data['test_interactions']
        self.train_weights = data['train_weights']

        self.model = LightFM(loss=self.loss, random_state=self.random_state)
        self.model.fit(self.train_interactions, epochs=self.epochs, sample_weight=self.train_weights, verbose=True)
        return self

    def transform(self, data):
        # Calcular AUC dentro de transform
        self.train_auc = auc_score(self.model, self.train_interactions, num_threads=4).mean()
        self.test_auc = auc_score(self.model, self.test_interactions, num_threads=4).mean()

        return {
            'model': self.model,
            'train_auc': self.train_auc,
            'test_auc': self.test_auc
        }

In [10]:
# pipeline completo
pipeline = Pipeline(steps=[
    ('Crear Dataset', crearDataset()),
    ('Matriz usurio Movie', Matriz_usurio_Movie()),
    ('Entrenar modelo lightfm', EntrenarModelo(loss='warp', epochs=20)) ## 'logistic', 'bpr', 'warp',
])
pipeline

Pipeline(steps=[('Crear Dataset', crearDataset()),
                ('Matriz usurio Movie', Matriz_usurio_Movie()),
                ('Entrenar modelo lightfm',
                 EntrenarModelo(epochs=20, loss='warp'))])

In [11]:
salidas =pipeline.fit_transform(tabla_completa)
#Area bajo la curva de train y test
print()
print(f"AUC de entrenamiento: {salidas['train_auc']:.2f}")
print(f"AUC de prueba: {salidas['test_auc']:.2f}")

Epoch: 100%|██████████| 20/20 [00:00<00:00, 57.09it/s]


AUC de entrenamiento: 0.86
AUC de prueba: 0.76


Se observa que el AUC en el conjunto de entrenamiento es de 0.86, lo que indica que el modelo tiene un buen desempeño al predecir las preferencias de los usuarios sobre los datos con los que fue entrenado. No obstante, el AUC en el conjunto de prueba es un poco menor al AUC de entrenamiento, lo cual  sugiere la presencia de cierto sobreajuste; es decir, el modelo se adapta mejor a los datos conocidos que a los nuevos. A pesar de esta diferencia, un AUC de 0.76 en prueba sigue siendo un resultado favorable.

<font color='056938'> **Visualización recomendación** </font>

In [29]:
# Acceder al dataset_train desde el pipeline
dataset_train = pipeline.named_steps['Crear Dataset'].dataset_train

# Acceder al modelo de LightFM desde el pipeline
model = pipeline.named_steps['lightfm_trainer'].model

In [33]:
def recommendacion(model, data, original_user_id, conn, k):
    #Se extraen de la base de datos todas las películas que han sido calificadas por otros usuarios (no el usuario actual)
    df_movies_no_vistas = pd.read_sql(f'SELECT * FROM full_ratings WHERE user_id <> {original_user_id}', conn)

    # Se obtienen los ID de las películas que NO ha calificado el usuario actual
    movieid_no_vistas = df_movies_no_vistas['movie_id'].values

    # Se identifican los ID de las películas no vistas por el usuario
    item_id_no_vista = [value for key, value in data.mapping()[2].items() if value not in movieid_no_vistas]

    # Se obtiene el ID del usuario transformado según el índice interno del modelo
    uid_index = data.mapping()[0][original_user_id]

    # Se predicen los puntajes para las películas no vistas por el usuario
    scores = model.predict(uid_index, item_id_no_vista)

    # Se ordenan los índices de los ítems según los puntajes, de mayor a menor
    sorted_indices = np.argsort(-scores).tolist()

    # Se obtienen los ID reales de las películas con los puntajes más altos
    top_items = [key for key, value in data.mapping()[2].items() if value in sorted_indices[:k]]

    # Se filtran las películas recomendadas con sus títulos y se eliminan duplicados
    recommended = df_movies_no_vistas[df_movies_no_vistas['movie_id'].isin(top_items)][['movie_id', 'movie_title']]
    recommended.drop_duplicates(inplace=True)

    return recommended

In [31]:
# Interfaz interactiva
def mostrar_recomendaciones(user_id, k):
    print(f"\n Top '{k}' de peliculas recomendadas para el usuario '{user_id}':")
    result = recommendacion(model, dataset_train, user_id, con, k)
    display(result)

# Obtener lista de usuarios válidos desde el mapping
user_ids = list(dataset_train.mapping()[0].keys())

interact(
    mostrar_recomendaciones,
    user_id=Dropdown(options=user_ids, description="Usuario"),
    k=IntSlider(min=1, max=20, step=1, value=5, description="Top K")
);

interactive(children=(Dropdown(description='Usuario', options=(np.int64(1), np.int64(4), np.int64(6), np.int64…

In [32]:
import nbformat

# Path to your notebook
input_notebook = '/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/4.Modelo_3.ipynb'
output_notebook = '/content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/4.Modelo_3 correcto.ipynb'

# Load the notebook
with open(input_notebook, 'r') as f:
    notebook = nbformat.read(f, as_version=4)

# Check if the notebook has 'metadata.widgets' and remove it
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']  # Removes widgets metadata completely

# Save the modified notebook
with open(output_notebook, 'w') as f:
    nbformat.write(notebook, f)

print(f"Fixed notebook saved as {output_notebook}")

Fixed notebook saved as /content/drive/MyDrive/Mod2/ANALITICA3/Trabajo Marketing/4.Modelo_3 correcto.ipynb
